## 1. Document ingestion

In this step, the notebook loads the paper and prepares it for downstream tasks. The code uses requests to grab the PDF and fitz (PyMuPDF) to walk through each page. Then the text on every page is split into fixed-size chunks so it can be embedded later, and any images on the pages are extracted and saved.


In [ ]:
!pip install PyMuPDF
import os
import io
import requests
import fitz
from PIL import Image




pdf_url = "https://arxiv.org/pdf/1706.03762.pdf"
pdf_path = "attention.pdf"


if not os.path.exists(pdf_path):
    response = requests.get(pdf_url, headers={"User-Agent": "Mozilla/5.0"})
    if response.status_code == 200:
        with open(pdf_path, "wb") as f:
            f.write(response.content)
    else:
        raise RuntimeError(f"Failed to download PDF (status {response.status_code}). Please download it manually and place it at {pdf_path}.")


doc = fitz.open(pdf_path)
chunks = []  # list of {page, text}
images = []  # list of {page, image}
chunk_size = 500
for page_num in range(doc.page_count):
    page = doc[page_num]
    text = page.get_text().strip()

    for i in range(0, len(text), chunk_size):
        chunk_text = text[i:i + chunk_size]
        chunks.append({"page": page_num, "text": chunk_text})

    for img_index, img_info in enumerate(page.get_images(full=True)):
        xref = img_info[0]
        base_image = doc.extract_image(xref)
        image_bytes = base_image["image"]
        pil_image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
        images.append({"page": page_num, "image": pil_image})

doc.close()

## 2. Embedding with Jina CLIP V1

For embeddings, the notebook uses Jina CLIP V1, a multimodal model that maps both text and images into a shared vector space so they can be compared directly. The model is loaded via the Hugging Face transformers API with trust_remote_code=True, which lets it expose a higher-level interface on top of the base CLIP architecture.


In [ ]:
import torch
from transformers import AutoModel
import numpy as np
from collections import defaultdict


clip_model = AutoModel.from_pretrained("jinaai/jina-clip-v1", trust_remote_code=True)
clip_model.eval()

USE_CUDA = torch.cuda.is_available()
USE_FP16 = False  # flip to True only if GPU hits memory limits

if USE_CUDA:
    clip_model = clip_model.to("cuda")
    if USE_FP16:
        clip_model = clip_model.half()


texts = [c["text"] for c in chunks]
image_list = [im["image"] for im in images] if images else []


page_to_images = defaultdict(list)
for im in images:
    page_to_images[im["page"]].append(im["image"])


def batched(seq, n):
    for i in range(0, len(seq), n):
        yield seq[i:i+n]


@torch.no_grad()
def encode_text(text_list):
    # Jina-CLIP remote code returns numpy arrays
    return clip_model.encode_text(text_list)

@torch.no_grad()
def encode_images(pil_images):
    return clip_model.encode_image(pil_images)


TEXT_BATCH = 64
IMG_BATCH = 16

text_embeds = []
for b in batched(texts, TEXT_BATCH):
    if b:
        text_embeds.append(encode_text(b))
text_embeddings = np.vstack(text_embeds) if text_embeds else np.zeros((0, 768), dtype=np.float32)

image_embeds = []
for b in batched(image_list, IMG_BATCH):
    if b:
        image_embeds.append(encode_images(b))
image_embeddings = np.vstack(image_embeds) if image_embeds else np.zeros((0, 768), dtype=np.float32)


def l2_normalize(a, eps=1e-12):
    if a.size == 0:
        return a
    norms = np.linalg.norm(a, axis=1, keepdims=True)
    return a / np.maximum(norms, eps)

text_embeddings = l2_normalize(text_embeddings)
image_embeddings = l2_normalize(image_embeddings)



## 3. Storing embeddings in Chroma

To persist the embeddings, the notebook uses the Chroma vector database, which is designed for fast similarity search over high‑dimensional vectors. It relies on approximate nearest‑neighbour indexing so that relevant chunks can be retrieved quickly even as the collection grows.


In [ ]:
!pip install chromadb
import chromadb
from math import ceil


db_client = chromadb.Client()

collection = db_client.get_or_create_collection(name="transformer_paper")


texts = [c["text"] for c in chunks]
text_ids = [f"text_{i}" for i in range(len(texts))]
text_metadatas = [{"page": c["page"], "type": "text", "idx": i} for i, c in enumerate(chunks)]


try:
    _ = text_embeddings
except NameError:
    text_embeddings = encode_text(texts)


text_vectors = [row.tolist() for row in text_embeddings]


BATCH = 512
for b in range(ceil(len(texts) / BATCH)):
    s = b * BATCH
    e = min((b + 1) * BATCH, len(texts))
    if s < e:
        collection.add(
            ids=text_ids[s:e],
            embeddings=text_vectors[s:e],
            metadatas=text_metadatas[s:e],
            documents=texts[s:e],
        )

if images:
    pil_images = [im["image"] for im in images]
    image_ids = [f"img_{i}" for i in range(len(images))]
    image_metadatas = [{"page": im["page"], "type": "image", "idx": i} for i, im in enumerate(images)]


    try:
        _ = image_embeddings
    except NameError:
        image_embeddings = encode_images(pil_images)

    image_vectors = [row.tolist() for row in image_embeddings]

    for b in range(ceil(len(pil_images) / BATCH)):
        s = b * BATCH
        e = min((b + 1) * BATCH, len(pil_images))
        if s < e:
            collection.add(
                ids=image_ids[s:e],
                embeddings=image_vectors[s:e],
                metadatas=image_metadatas[s:e],
                documents=[""] * (e - s),
            )

In [ ]:
print(collection.count())

In [ ]:
len(texts)+len(images)

## 4. Retrieval

For retrieval, the notebook queries the Chroma collection by first embedding the query string using Jina CLIP. The resulting embedding is then used to find the top-k nearest neighbours in the vector database.​

The retrieval function returns the matching documents, their associated metadata (like page or chunk ID), and the distance scores that indicate how closely each result matches the query. This allows for both fast similarity search and easy tracing back to the original context.


In [ ]:
import numpy as np

def _l2_normalize(v, eps=1e-12):
    if v.ndim == 1:
        n = np.linalg.norm(v) or eps
        return v / max(n, eps)
    n = np.linalg.norm(v, axis=1, keepdims=True)
    return v / np.maximum(n, eps)

def retrieve(query, top_k=3, where=None):
    """
    Retrieve most relevant text chunks for a query.
    - Normalizes query embedding to match index normalization.
    - Optional `where` filter; defaults to text-only.
    """
    if where is None:
        where = {"type": "text"}


    q_emb = encode_text([query])[0]
    q_emb = _l2_normalize(q_emb)


    count = collection.count()
    k = min(top_k, max(count, 0))

    if k == 0:
        return {"documents": [], "metadatas": [], "distances": []}

    res = collection.query(
        query_embeddings=[q_emb.tolist()],
        n_results=k,
        include=["documents", "metadatas", "distances"],
        where=where
    )
    return res

def show_results(res, max_chars=160):
    """Pretty-print results for quick sanity checks."""
    if not res or not res.get("documents"):
        return
    docs = res["documents"][0]
    metas = res["metadatas"][0]
    dists = res["distances"][0]
    for i, (doc, meta, dist) in enumerate(zip(docs, metas, dists), 1):
        snippet = (doc[:max_chars] + "…") if doc and len(doc) > max_chars else (doc or "")
        page = meta.get("page", "?")


## 5. Generation with Phi-3 Vision

The notebook uses the Phi-3 Vision model for multimodal generation, which can process both a single image and a long text prompt in a special chat format. The input should be structured like this:

```
<|user|>
<|image_1|>
{prompt}<|end|>
<|assistant|>

```

A small blank image is supplied if no relevant image was retrieved.


In [ ]:

import os, gc, torch
import numpy as np
from transformers import AutoModelForCausalLM, AutoProcessor
from PIL import Image

# ---------------------------
# 0) Clean up prior models if cell re-run
# ---------------------------
if 'phi3_model' in globals():
    try:
        phi3_model.to('cpu')
    except Exception:
        pass
    try:
        del phi3_model
    except Exception:
        pass
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# ---------------------------
# 1) Load Phi-3 Vision (robust hybrid offload)
# ---------------------------
phi3_id   = "microsoft/Phi-3-vision-128k-instruct"
cuda_cap  = "12GiB"
off_dir   = "offload_phi3"
os.makedirs(off_dir, exist_ok=True)

def _clear_cuda():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

# Try multiple key styles for max_memory to satisfy different HF/Accelerate versions
def _try_load_phi3_with_keys():
    gpu_idx = 0
    try:
        if torch.cuda.is_available():
            gpu_idx = torch.cuda.current_device()
    except Exception:
        pass

    candidate_keys = [
        gpu_idx,                   # e.g., 0 (int)
        f"cuda:{gpu_idx}",         # e.g., "cuda:0"
        f"{gpu_idx}",              # e.g., "0"
        "cuda",                    # generic
    ]

    last_err = None
    for key in candidate_keys:
        try:
            model = AutoModelForCausalLM.from_pretrained(
                phi3_id,
                trust_remote_code=True,
                torch_dtype=(torch.float16 if torch.cuda.is_available() else "auto"),
                device_map="auto",
                max_memory={key: cuda_cap, "cpu": "64GiB"},
                offload_folder=off_dir,
                low_cpu_mem_usage=True,
                _attn_implementation="eager",
            )
            return model
        except Exception as e:
            last_err = e
            _clear_cuda()


    try:
        model = AutoModelForCausalLM.from_pretrained(
            phi3_id,
            trust_remote_code=True,
            torch_dtype=(torch.float16 if torch.cuda.is_available() else "auto"),
            device_map="auto",
            offload_folder=off_dir,
            low_cpu_mem_usage=True,
            _attn_implementation="eager",
        )
        return model
    except Exception as e:
        raise last_err or e

phi3_model = _try_load_phi3_with_keys()
phi3_model.eval()

phi3_processor = AutoProcessor.from_pretrained(phi3_id, trust_remote_code=True)
if getattr(phi3_processor.tokenizer, "pad_token_id", None) is None:
    phi3_processor.tokenizer.pad_token = phi3_processor.tokenizer.eos_token

blank_image = Image.new("RGB", (224, 224), color=(255, 255, 255))

# ---------------------------
# 2) Utilities
# ---------------------------
def _truncate_context(context: str, max_chars: int = 6000) -> str:
    return context[:max_chars]

def _l2_normalize(v, eps=1e-12):
    v = np.asarray(v, dtype=np.float32)
    n = np.linalg.norm(v)
    return v / max(n, eps)


try:
    page_to_images
except NameError:
    from collections import defaultdict
    page_to_images = defaultdict(list)
    try:
        for im in images:
            page_to_images[im["page"]].append(im["image"])
    except Exception:
        pass

# ---------------------------
# 3) Image selection
# ---------------------------
def select_images_for_query(query: str, top_k_images: int = 3, prefer_text_pages: bool = True):
    selected = []


    if prefer_text_pages:
        try:
            res = retrieve(query, top_k=5, where={"type": "text"})
            metas = (res or {}).get("metadatas") or []
            if metas and isinstance(metas[0], list):
                metas = metas[0]
            pages = [m.get("page") for m in metas if isinstance(m, dict) and "page" in m]
            for p in pages:
                for img in page_to_images.get(p, []):
                    if len(selected) < top_k_images:
                        selected.append(img)
        except Exception:
            pass


    if len(selected) < 1:
        try:
            q = encode_text([query])[0]
            q = _l2_normalize(q)
            res_img = collection.query(
                query_embeddings=[q.tolist()],
                n_results=top_k_images,
                include=["metadatas"],
                where={"type": "image"}
            )
            metas = (res_img or {}).get("metadatas") or []
            if metas and isinstance(metas[0], list):
                metas = metas[0]
            for m in metas:
                p = m.get("page")
                if p in page_to_images:
                    for img in page_to_images[p]:
                        if len(selected) < top_k_images:
                            selected.append(img)
        except Exception:
            pass

    return selected

# ---------------------------
# 4) Core: generate_answer (Vision-only) with self-cleanup
# ---------------------------
def generate_answer(
    query: str,
    top_k: int = 3,
    max_tokens: int = 200,
):
    """
    Vision-only Phi-3 answer generation with real PDF figures.
    Frees CUDA memory at the end of the call.
    """
    import gc as _gc
    inputs = None
    output_ids = None
    imgs = None
    try:

        hits = retrieve(query, top_k=top_k) or {}
        docs = hits.get("documents") or []

        # Flatten list-of-lists -> list[str]
        context_chunks = [d for lst in docs for d in (lst if isinstance(lst, list) else [lst]) if isinstance(d, str)]
        context = "\n\n".join(context_chunks) if context_chunks else "No relevant context."
        context = _truncate_context(context)

        # Select up to 3 relevant images
        imgs = select_images_for_query(query, top_k_images=3, prefer_text_pages=True)
        if not imgs:
            imgs = [blank_image]

        # Build chat with matching <|image_i|> tags
        image_tags = " ".join(f"<|image_{i+1}|>" for i in range(len(imgs)))
        messages = [{
            "role": "user",
            "content": f"{image_tags}\nContext:\n{context}\n\nQuestion: {query}\nAnswer based solely on the context."
        }]
        prompt = phi3_processor.tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

        # Tokenize + move to model device
        inputs = phi3_processor(text=prompt, images=imgs, return_tensors="pt")
        device = next(phi3_model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate (use_cache=False avoids DynamicCache issue)
        with torch.no_grad():
            output_ids = phi3_model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                do_sample=False,
                temperature=0.0,
                eos_token_id=phi3_processor.tokenizer.eos_token_id,
                pad_token_id=phi3_processor.tokenizer.pad_token_id,
                use_cache=False,
            )

        # Strip prompt and decode
        output_ids = output_ids[:, inputs["input_ids"].shape[1]:]
        response = phi3_processor.batch_decode(
            output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )[0].strip()
        return response

    finally:
        # drop large tensors and clear cache each call
        try:
            del output_ids
        except Exception:
            pass
        try:
            if isinstance(inputs, dict):
                for k in list(inputs.keys()):
                    try: del inputs[k]
                    except Exception: pass
            del inputs
        except Exception:
            pass
        try:
            del imgs
        except Exception:
            pass
        _gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

# (Optional) quick sanity info
#try:
#    from pprint import pprint
#except Exception:
#    pass


In [ ]:
# Minimal smoke test
answer = generate_answer(
    "What is the main idea of the Transformer paper?",
    top_k=1,
    max_tokens=60
)
print(answer)


In [ ]:
answer = generate_answer(
    "What is the purpose of positional encoding in the Transformer model?",
    top_k=2,
    max_tokens=80
)
print(answer)


In [ ]:
# This prompt requires a GPU with higher VRAM (an A100 works well).
answer = generate_answer(
    "Describe the sinusoidal positional encoding figure and what the color bands mean.",
    top_k=2,
    max_tokens=120
)
print(answer)


## 6. Conclusion

This notebook shows how to build a multimodal retrieval-augmented generation (RAG) pipeline step by step, without depending on high-level frameworks. The workflow includes parsing a PDF, embedding both text and figures using Jina CLIP, storing and retrieving vectors with ChromaDB, and finally generating grounded answers with Phi-3 Vision.